In [1]:
%reload_ext autoreload
%autoreload 2
os.chdir("..")
import toml
from core.dl_framework.utils import read_config

/usr/local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%%bash 
pwd

/content/Template_Project


In [3]:
config_file = toml.load("configs/default_train_config.toml")
setup_config = read_config(config_file)

In [5]:
from core.dl_framework.learner import Learner
from core.dl_framework.callbacks import Recorder, Monitor, EarlyStopping, Checkpoints
from core.dl_framework.data import get_dataset, Dataset, DataLoader, split_data
from torchvision import transforms
bs = setup_config["h_batch_size"]
x_train, y_train, x_test, y_test = get_dataset(setup_config["s_source"], setup_config["s_set"], setup_config["p_tmp_data_path"], CNN=True)

train_ds, test_ds =  Dataset(x_train, y_train), Dataset(x_test, y_test)
train_dl, test_dl = DataLoader(train_ds, bs), DataLoader(test_ds, bs)
learn = Learner(train_ds, setup_config)
learn.fit(2)


100%|██████████| 187/187 [00:00<00:00, 1171.62it/s]


epoch: 1/2	[valid_acc: 0.8958	valid_loss: 0.6926	train_loss: 1.2775]
new best: valid_loss 0.6926330876222906
new checkpoint


100%|██████████| 187/187 [00:00<00:00, 1180.10it/s]

epoch: 2/2	[valid_acc: 0.9357	valid_loss: 0.3742	train_loss: 0.4922]
new best: valid_loss 0.3742092729092919
new checkpoint


In [4]:
from pathlib import Path
from datetime import datetime
ckp_path = "/content/gdrive/MyDrive/Marcel_Moczarski/Data_Science/Google_Services/Local_Drive_Storage/Conda_Envs/Template_Project/checkpoints"

In [169]:
sample = next(iter(test_dl))
imgs, targets = sample

In [90]:
from pathlib import Path
import os
ckp_path = Path(setup_config["c_c_ckp_path"])

def get_history(ckp_path, monitor, fileformat=["csv"]):
    project_name = Path(os.getcwd()).name
    for i, path in enumerate(ckp_path.parts):
        length = len(ckp_path.parts)
        if (path == project_name) and (i+1 != length):
            project_path = ckp_path.parents[length-i-2]

    files = [] 
    if type(fileformat) != list: 
        fileformat = [fileformat]
    for fmt in fileformat:
        for rel_path in ckp_path.rglob("*."+fmt):
            # files.append(rel_path.relative_to(project_path))
            files.append(rel_path)
        metric_files = []
        for metric in files:
            if monitor in metric.name:
                metric_files.append(metric)
            
    return metric_files

def get_specific_history(ckp_path, monitor, fileformat=["csv"], specific="best"):
    files = get_history(ckp_path, monitor, fileformat)

    if specific == "best":
        best_vals = []
        for idx, hist in enumerate(files):
            get_func = getattr(pd, "read_" + hist.suffix[1:])
            tmp_df = get_func(hist)
            best_vals.append(tmp_df[monitor].max())
        best_idx = best_vals.index(max(best_vals))
        # best_hist = 
        return getattr(pd, "read_" + files[best_idx].suffix[1:])(files[best_idx])

best_hist = get_specific_history(ckp_path, "valid_acc", ["parquet", "csv"], "best")
best_hist

,valid_acc,valid_loss,train_loss
epochs,,,
1,0.892463,0.694473,1.296461
2,0.935244,0.377675,0.496537
3,0.951454,0.260701,0.304477
4,0.958640,0.199920,0.221037
5,0.964238,0.162641,0.174001
...,...,...,...
97,0.991143,0.031304,0.001217
98,0.991143,0.031336,0.001146
99,0.991143,0.031410,0.001079


In [45]:
import pandas as pd
import numpy as np

arr_size = 1e2
arr_cols = "ABCD"
tmp_dict = {}

fileformat = "parquet"
arr = np.arange(arr_size)

for letter in arr_cols:
    tmp_dict[letter] = arr

df = pd.DataFrame(tmp_dict)

to_func = getattr(df, "to_"+fileformat)
to_func(ckp_path / Path(f"test.{fileformat}"))


In [136]:
# import plotly.io as pio
# pio.reanderers.default = "notebook_connected"
import itertools

In [177]:
# !pip install plotly
import plotly.io as pio
import itertools
pio.renderers.default = "notebook_connected"
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
col_pal = px.colors.sequential.Rainbow
col_pal_iter = itertools.cycle(col_pal) 
# def print_history(history):
history = best_hist

implemented_metric = ["acc", "loss"]
plot_dict = {}
for met in implemented_metric:
    plot_dict[met] = [s for s in history if met in s]

fig = make_subplots(rows=1, cols=len(plot_dict), subplot_titles=list(plot_dict.keys()))

for idx, (met, mets) in enumerate(plot_dict.items()):
    for key in mets:
        new_color = next(col_pal_iter)
        new_color = next(col_pal_iter)
        fig.add_trace(go.Scatter(x=history.index, y=history[key].values, name=key, line=dict(color=new_color)), row=1, col=idx+1)
        dmin, dmax = history[key].min(), history[key].max()
        if met == "acc":
            epoch = history[key].argmax()
            fig.add_trace(go.Scatter(x=[epoch], y=[dmax], line=dict(color=new_color, width=1, dash="dash")), row=1, col=idx+1)
            fig.add_vline(x=epoch, line_width=1, line_color=new_color, line_dash="dash", row=1, col=idx+1)
        if met == "loss":
            epoch = history[key].argmin()
            fig.add_trace(go.Scatter(x=[epoch], y=[dmin], line=dict(color=new_color, width=1, dash="dash")), row=1, col=idx+1)
            fig.add_vline(x=epoch, line_width=1, line_color=new_color, line_dash="dash", row=1, col=idx+1)
    # break
# learn.history.keys()[0]
# fig.update_layout(title_text="test")
fig.show()

In [17]:
list(plot_dict.keys())

['acc', 'loss']

In [22]:
globals()["test_class"]


__main__.test_class

In [29]:
%%timeit
test_num.max()

1.57 µs ± 284 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [41]:
%%timeit
max(test_dict["test"])


281 ns ± 10.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [47]:
import types
class test_class():
    def __init__(self, a):
        self.attr = a
    
    def attr(self):
        pass

test = test_class(1)

getattr(test_class, "attr")

<function __main__.test_class.attr(self)>

In [35]:
%%timeit
test1.sum()

2.09 µs ± 52.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [32]:
import numpy as np

In [16]:
class test1:
    def __init__(self, a):
        self.a = a
    def test_func(self):
        return self.a
a = test1(1)
setattr(a, "s", 123)
vars(a)

{'a': 1, 's': 123}

In [17]:
monitor = Monitor_Cb(["val_acc"])
early = EarlyStopping_Cb(monitor="val_loss")
recorder = Recorder_Cb("hi")
cbh = CallbackHandler([monitor, early, recorder])
cbh.Monitor_Cb.learn


NameError: name 'Monitor_Cb' is not defined

In [9]:
a = ["a"]
for i in a[:4]:
    print(i)

a
